# 处理数据

数据是构建机器学习模型的基础。在云中集中管理数据，并使在多个工作站上运行试验和训练模型的数据科学家团队能够访问这些数据以及计算目标，这是任何专业数据科学解决方案的重要组成部分。

在该笔记本中，你将探索两个用于数据处理的 Azure 机器学习对象：数据存储和数据集****。

## 连接到工作区

首先，请连接到你的工作区。

> 备注****：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 使用数据存储

在 Azure ML 中，数据存储是对存储位置（例如 Azure 存储 Blob 容器）的引用**。每个工作区都有一个默认的数据存储，该存储通常是使用相应工作区创建的 Azure 存储 Blob 容器。如果需要使用存储在不同位置的数据，则可以将自定义数据存储添加到工作区中，并将其中任何一个设置为默认值。

### 查看数据存储

运行以下代码以确定工作区中的数据存储：

In [ ]:
# 获取默认数据存储
default_ds = ws.get_default_datastore()

# 枚举所有数据存储，指出默认项
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

你还可以在 [Azure 机器学习工作室](https://ml.azure.com)中工作区的“数据集”页面上查看和管理工作区中的数据存储****。

### 将数据上传到数据存储

确定可用的数据存储后，可以将文件从本地文件系统上传到数据存储，这样无论试验脚本实际在何处运行，工作区中运行的试验都能访问相应文件。

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

## 使用数据集

Azure 机器学习以数据集的形式提供数据的抽象**。数据集是对可能要在试验中使用的一组特定数据的版本控制引用。数据集可以采用表格格式，也可以采用文件格式****。

### 创建表格数据集

接下来根据上传到数据存储的糖尿病数据创建数据集，然后查看前 20 条记录。这种情况下，数据在 CSV 文件中采用结构化格式，因此我们将使用表格数据集**。

In [ ]:
from azureml.core import Dataset

# 获取默认数据存储
default_ds = ws.get_default_datastore()

#从数据存储上的路径创建表格数据集（这可能需要一些时间）
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 将前 20 行显示为 Pandas 数据帧
tab_data_set.take(20).to_pandas_dataframe()

如上述代码中所见，可以轻松地将表格数据集转换为 Pandas 数据帧，从而使用常见的 python 技术处理数据。

### 创建文件数据集

你创建的数据集是表格数据集，可以在数据集定义所包含的结构化文件中作为包含所有数据的数据帧读取**。这对于表格数据非常有效，但在某些机器学习场景中，可能需要使用非结构化数据；或者你可能只想通过自己的代码读取文件中的数据。为此，可以使用文件数据集，该数据集在虚拟装入点创建文件路径列表，用于读取文件中的数据**。

In [ ]:
#从数据存储上的路径创建文件数据集（这可能需要一些时间）
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# 获取数据集中的文件
for file_path in file_data_set.to_path():
    print(file_path)

### 注册数据集

创建引用糖尿病数据的数据集后，可以将其注册，确保工作区中运行的所有试验可轻松对其进行访问。

我们将表格数据集注册为“糖尿病数据集”，将文件数据集注册为“糖尿病文件”********。

In [ ]:
# 注册表格数据集
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diabetes dataset',
                                        description='diabetes data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# 注册文件数据集
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name='diabetes file dataset',
                                            description='diabetes files',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

你可以在 [Azure 机器学习工作室](https://ml.azure.com)中工作区的“数据集”页面上查看和管理数据集****。你还可以从工作区对象获取数据集列表：

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

通过对数据集进行版本控制，可以重新定义数据集，从而无需破坏依赖先前定义的现有试验或管道。默认返回最新版本的已命名数据集，但可以通过指定版本号检索特定版本的数据集，如下所示：

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


### 从表格数据集训练模型

有数据集后，即可开始从中训练模型。可以在运行脚本的估算器中将数据集作为输入传递给脚本**。

运行以下两个代码单元格，创建以下内容：

1.名为 diabetes_training_from_tab_dataset 的文件夹****
2.使用传递给它的表格数据集训练分类模型的脚本作为参数。

In [ ]:
import os

# 为试验文件创建文件夹
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import os
import argparse
from azureml.core import Run, Dataset
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 获取脚本参数（正则化率和训练数据集 ID）
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# 设置正则化超参数（作为参数传递给脚本）
reg = args.reg_rate

# 获取试验运行上下文
run = Run.get_context()

# 获取训练数据集
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练逻辑回归模型
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 注意，保存在 outputs 文件夹中的文件会自动上传到试验记录
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

> 备注****：在脚本中，数据集作为形参（或实参）传递。对于表格数据集，此参数将包含已注册数据集的 ID，因此，你可以在脚本中编写代码以从运行上下文中获取试验的工作区，然后使用其 ID 获取数据集，如下所示：
>
> ```
> run = Run.get_context()
> ws = run.experiment.workspace
> dataset = Dataset.get_by_id(ws, id=args.training_dataset_id)
> diabetes = dataset.to_pandas_dataframe()
> ```
>
> 但是，Azure 机器学习运行时会自动识别引用命名数据集的参数并将其添加到运行的 input_datasets 集合中，因此你还可以通过指定其“易记名称”来从该集合检索数据集（稍后你将看到，它在试验的脚本运行配置中的参数定义中指定）****。这是上面脚本中采用的方法。

现在，可以试验方式运行脚本，为训练数据集定义一个由脚本读取的参数。

> 备注****：Dataset 类取决于 azureml-dataprep 包中的某些组件，该组件包括对 to_pandas_dataframe() 方法使用的 Pandas 的可选支持****************。因此，需要在将要运行训练试验的环境中包含此包。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 创建用于试验的 Python 环境
sklearn_env = Environment("sklearn-env")

# 确保已安装所需的包（我们需要 scikit-learn、Azure ML 默认值和 Azure ML dataprep）
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# 获取训练数据集
diabetes_ds = ws.datasets.get("diabetes dataset")

# 创建脚本配置
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=sklearn_env) 

# 提交试验
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

> **备注**：--input-data 参数将数据集作为指定输入传递，其中包括该数据集的易记名称，脚本在试验运行中使用该名称从 input_datasets 集合读取它************。--input-data 参数中的字符串值实际上是已注册数据集的 ID****。  作为一种替代方法，可以只传递 `diabetes_ds.id`，在这种情况下，脚本可以从脚本参数访问数据集 ID，并使用该 ID 从工作区（而不是从 input_datasets 集合）获取数据集****。

首次运行试验时，可能需要一些时间来设置 Python 环境 - 后续运行会更快。

试验完成后，在小组件中查看 azureml-logs/70_driver_log.txt 输出日志和运行所生成的指标****。

### 注册训练后的模型

与任何训练试验一样，可以检索训练后的模型并在 Azure 机器学习工作区中注册它。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Tabular dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

### 从文件数据集训练模型

你已了解了如何使用表格数据集中的训练数据来训练模型。但文件数据集呢****？

使用文件数据集时，传递给脚本的数据集参数表示包含文件路径的装入点。从这些文件中读取数据的方式取决于文件中的数据类型及其预期用途。对于糖尿病 CSV 文件，可以使用 Python glob 模块在数据集定义的虚拟装入点中创建文件列表，并将其全部读入可联结为单个数据帧的 Pandas 数据帧中****。

运行以下两个代码单元格，创建以下内容：

1.名为 diabetes_training_from_file_dataset 的文件夹****
2.使用文件数据集（作为输入传递给脚本）训练分类模型的脚本**。

In [ ]:
import os

# 为试验文件创建文件夹
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import os
import argparse
from azureml.core import Dataset, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# 获取脚本参数（正则化率和文件数据集装入点）
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--input-data', type=str, dest='dataset_folder', help='data mount point')
args = parser.parse_args()

# 设置正则化超参数（作为参数传递给脚本）
reg = args.reg_rate

# 获取试验运行上下文
run = Run.get_context()

# 加载糖尿病数据集
print("Loading Data...")
data_path = run.input_datasets['training_files'] # Get the training data path from the input
# （如果不想依赖硬编码的易记名称，也可以只使用 args.dataset_folder）

# 读取文件
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练逻辑回归模型
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 注意，保存在 outputs 文件夹中的文件会自动上传到试验记录
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

与表格数据集一样，你可以使用其易记名称从 input_datasets 集合中检索文件数据集****。还可以从脚本参数检索它，对于文件数据集，脚本参数包含文件的安装路径（而不是传递给表格数据集的数据集 ID）。

接下来需要更改将数据集传递到脚本的方式 - 这需要定义脚本可以从中读取文件的路径。可以使用 as_download 或 as_mount 方法来执行此操作********。使用 as_download 会将文件数据集中的文件下载到计算机上运行脚本的临时位置，而 as_mount 会创建一个装入点，可以从该装入点直接从数据集传输文件********。

可以将访问方法与 as_named_input 方法结合使用，以在试验运行中将数据集包含在 input_datasets 集合中（如果不这样做，例如，通过将参数设置为 `diabetes_ds.as_mount()`，则脚本将能够从脚本参数（而不是从 input_datasets 集合）访问数据集装入点）************。

In [ ]:
from azureml.core import Experiment
from azureml.widgets import RunDetails


# 获取训练数据集
diabetes_ds = ws.datasets.get("diabetes file dataset")

# 创建脚本配置
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_files').as_download()], # Reference to dataset location
                                environment=sklearn_env) # Use the environment created previously

# 提交试验
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

试验完成后，在小组件中查看 azureml-logs/70_driver_log.txt 输出日志，以验证文件数据集中的文件已下载到临时文件夹中，从而使脚本能够读取文件****。

### 注册训练后的模型

同样，可以注册由试验训练的模型。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'File dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> 详细信息****：有关使用数据集进行训练的详细信息，请参阅 Azure ML 文档中的[使用数据集进行训练](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)。